In [3]:
import numpy as np
import re
from tensorflow.keras.layers import Dense, LSTM, Input, Dropout, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
f = open('C:\\Users\\anast\\jupyter_notebook\\Anastasia_Notes\\Project TripAdvisor\\train_data_true.txt', 'r', encoding='utf-8')
texts_true = f.readlines()
texts_true[0] = texts_true[0].replace('\ufeff','')

In [5]:
f = open('C:\\Users\\anast\\jupyter_notebook\\\Anastasia_Notes\\Project TripAdvisor\\train_data_false.txt', 'r', encoding='utf-8')
texts_false = f.readlines()
texts_false[0] = texts_false[0].replace('\ufeff','')

In [6]:
texts = texts_true + texts_false
count_true = len(texts_true)
count_false = len(texts_false)
total_lines = count_true + count_false
print(count_true,count_false,total_lines)

110 109 219


In [7]:
maxWordsCount = 2000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
tokenizer. fit_on_texts (texts)

In [8]:
data = tokenizer.texts_to_sequences(texts)
max_text_len = 10
data_pad = pad_sequences(data, maxlen=max_text_len)
print(data_pad)
print(data_pad.shape)

[[ 226   57    2 ...  228  229  230]
 [   0    5  231 ...  234    3   77]
 [ 235    3   77 ...    5  237  238]
 ...
 [   0    0    0 ... 1048    2 1049]
 [   0    0    0 ...  196 1050 1051]
 [   0    0    0 ...    8  189  215]]
(219, 10)


In [9]:
X = data_pad
Y = np.array([[1, 0]]*count_true + [[0, 1]]*count_false)
print(X.shape, Y.shape)

(219, 10) (219, 2)


In [10]:
indeces = np.random.choice(X.shape[0], size=X.shape[0], replace=False)
X = X[indeces]
Y = Y[indeces]

In [11]:
model = Sequential()
model.add(Embedding(maxWordsCount, 128, input_length = max_text_len))
model.add(LSTM(64, activation='tanh', return_sequences=True))
model.add(LSTM(32, activation='tanh'))
model.add(Dense(2, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', metrics = ['accuracy'], optimizer = Adam(0.0001))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 128)           256000    
                                                                 
 lstm (LSTM)                 (None, 10, 64)            49408     
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dense (Dense)               (None, 2)                 66        
                                                                 
Total params: 317,890
Trainable params: 317,890
Non-trainable params: 0
_________________________________________________________________


In [12]:
history = model.fit(X, Y, batch_size=32, epochs=50)

Epoch 1/50
7/7 [==============================] - 2s 8ms/step - loss: 0.6910 - accuracy: 0.6073
Epoch 2/50
7/7 [==============================] - 0s 9ms/step - loss: 0.6891 - accuracy: 0.6347
Epoch 3/50
7/7 [==============================] - 0s 9ms/step - loss: 0.6868 - accuracy: 0.6667
Epoch 4/50
7/7 [==============================] - 0s 8ms/step - loss: 0.6845 - accuracy: 0.7215
Epoch 5/50
7/7 [==============================] - 0s 9ms/step - loss: 0.6818 - accuracy: 0.7580
Epoch 6/50
7/7 [==============================] - 0s 9ms/step - loss: 0.6783 - accuracy: 0.7717
Epoch 7/50
7/7 [==============================] - 0s 8ms/step - loss: 0.6746 - accuracy: 0.7808
Epoch 8/50
7/7 [==============================] - 0s 9ms/step - loss: 0.6692 - accuracy: 0.8174
Epoch 9/50
7/7 [==============================] - 0s 9ms/step - loss: 0.6630 - accuracy: 0.8128
Epoch 10/50
7/7 [==============================] - 0s 9ms/step - loss: 0.6547 - accuracy: 0.8174
Epoch 11/50
7/7 [======================

In [13]:
!pip install chardet
!pip install pandas

In [40]:
df = pd.read_csv('ready_to_sentiment.csv')
df = df.drop(columns=['Unnamed: 0'])

In [47]:
df['collected_reviews'][0]

"['плюс красивый заповедный зона просыпаться вид море засыпать шум волна пожалуй самый необычный наш отпуск вроде жить палатка минимальный набор необходимый комфортный пребывание идеальный вариант мечтать сходить поход палатка перспектива ходить туалет яма ёлка точно d. расслаблять помогать отдохнуть городской суета ярко минус готовый глэмпинг тяжело добраться неподготовленный человек добраться способ пешком моторный лодка лодка стоить поездка конец глэмпинг причал деревянный помост поэтому лодка прийтись прыгать вода забираться лодка вода', 'ездить глэмпинг добрый компания конец апрель поскольку апрель действительно выдаться холодный делать больший скидка откинуть наш погодный невезение сильно разочаровать соотношение', 'плюс красивый заповедный зона потрясать закат вкусный мидия блинчик завтрак приятный вежливый молодой человек который собственно приносить завтрак минус везде грязненький бросаться глаз жутко неудобный скрипучий кровать который спина болеть неделя душа намёк хотя еле 

In [42]:
df2 = df.copy()
# for p in range(30):
df2.iat[0,1][0]
    

'['

In [33]:
strings = df.shape[0]
strings

30

In [34]:
def res_to_string(result):
    count_all_positive = 0
    count_all_negative = 0
    if (result[0][0] < result[0][1]):
        count_all_negative += 1;
    elif (result[0][0]) > (result[0][1]):
        count_all_positive += 1;
    return [count_all_positive,count_all_negative]

In [36]:
tup_pos = []
tup_neg = []

for i in range(strings):
    a = 0
    b = 0
    for j in range (strings):
        t = df.iat[i,1][j]
        data = tokenizer.texts_to_sequences([t])
        data_pad = pad_sequences(data, maxlen = max_text_len)
        res = model.predict(data_pad)
        a += res_to_string(res)[0]
        b += res_to_string(res)[1]
    tup_pos.append(a)
    tup_neg.append(b)


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 15ms/step


KeyboardInterrupt: 

In [208]:
new['Number of positive reviews'] = tup_pos
new['Number of negative reviews'] = tup_neg
new

,places,collected_reviews,Number of positive reviews,Number of negative reviews
0,"Глэмпинг Доброе,","[плюсы: очень красивая заповедная зона, просып...",3,2
1,"Глэмпинг на озере Сиг,",[отдыхала одна с двумя детьми. очень повезло с...,4,1
2,"Глэмпинг Видно Озеро,","[забронировал проживание в глэмпинге , случайн...",0,2
3,"Глэмпинг Vezzika,",[плюсы: - хорошее и красивое местоположение ми...,3,1
4,"ЛЕС Глэмпинг и спа,","[решили мы посетить это чудесное место , забро...",3,2
5,"Глэмпинг NewCamp,","[глэмпинг находится близ посёлка эссойла, в со...",1,0
6,"Urman Camp Глэмпинг,",[приехали в глэмпинг к 5 попросили заранее зат...,5,0
7,"Велна Эко Спа Резорт,","[мне есть с чем сравнивать, поэтому посетив за...",3,2
8,"Глэмпинг Под небом,","[отличные условия проживания, очень отзывчивый...",4,1
9,"Гуляй-Город,",[приехали сегодня после дороги и музея в серпу...,5,0


In [226]:
for i in range(30):
    if new.iat[i,1] == ['']:
        new.iat[i,3] = new.iat[i,2]
new

,places,collected_reviews,Number of positive reviews,Number of negative reviews
0,"Глэмпинг Доброе,","[плюсы: очень красивая заповедная зона, просып...",3,2
1,"Глэмпинг на озере Сиг,",[отдыхала одна с двумя детьми. очень повезло с...,4,1
2,"Глэмпинг Видно Озеро,","[забронировал проживание в глэмпинге , случайн...",0,2
3,"Глэмпинг Vezzika,",[плюсы: - хорошее и красивое местоположение ми...,3,1
4,"ЛЕС Глэмпинг и спа,","[решили мы посетить это чудесное место , забро...",3,2
5,"Глэмпинг NewCamp,","[глэмпинг находится близ посёлка эссойла, в со...",1,0
6,"Urman Camp Глэмпинг,",[приехали в глэмпинг к 5 попросили заранее зат...,5,0
7,"Велна Эко Спа Резорт,","[мне есть с чем сравнивать, поэтому посетив за...",3,2
8,"Глэмпинг Под небом,","[отличные условия проживания, очень отзывчивый...",4,1
9,"Гуляй-Город,",[приехали сегодня после дороги и музея в серпу...,5,0
